In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import random
import os

print(torch.get_num_threads())
print(torch.get_num_interop_threads())

32
32


In [2]:
#x_train_orig = np.arange(-2.4, 2.8, 0.4)
#x_train_orig = np.arange(-2.4, 2.6, 0.2)
#x_train_orig = np.arange(-2.4, 2.5, 0.1)
#x_train_orig = np.arange(-2.4, 2.45, 0.05)
x_train_orig = np.arange(-2.4, 2.425, 0.025)

In [3]:
class FullModel(torch.nn.Module):
    def __init__(self):
        super(FullModel, self).__init__()
        self.fc1 = torch.nn.Linear(1, 8)
        self.fc2 = torch.nn.Linear(8, 8)
        self.fc3 = torch.nn.Linear(8, 1)
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

In [4]:
def func(x):
    x = x.astype(float)
    return np.power(x, 1) * np.power(np.sin(1.0 / 3.0 * x), 2)

def avg_l2_diff(y1, y2):
    return np.average(np.power(y1-y2, 2))

In [5]:
def train(model, optimizer, criterion, epoch):
    model.train()
    
    running_loss  = torch.tensor(0.0)

    for batch_idx, current_batch in enumerate(train_loader):     
        inp, current_batch_y = current_batch[0],        current_batch[1]

        optimizer.zero_grad()
        output = model(inp)
        gndtruth = current_batch_y

        loss = criterion(output, gndtruth)
        loss.backward()
        optimizer.step()
        running_loss  += loss.item()

    running_loss  = running_loss  / len(train_loader)
    
    if epoch % 40 == 0:
        print("Epoch: {}, Average loss: {:15.8f}".format(epoch, running_loss))

In [6]:
l2_diff_tot_before = []
l2_diff_tot_after = []
for seed in np.arange(20):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.use_deterministic_algorithms(True)
    
    model = FullModel()

    x_train = x_train_orig
    print("Orig train size = ", x_train.shape[0])
    nsample = x_train.shape[0] - 1

    y_train = func(x_train)
    x_train_torch = torch.from_numpy(x_train).float()
    y_train_torch = torch.from_numpy(y_train).float()
    train_dataset = torch.utils.data.TensorDataset(x_train_torch, y_train_torch)
    train_loader = torch.utils.data.DataLoader(
        train_dataset, shuffle=True)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.02)
    criterion = torch.nn.MSELoss()
    
    for epoch in range(1, 2000):
        train(model, optimizer, criterion, epoch)
        
    x_pred_np = np.arange(-2.5, 2.5001, 0.0001).reshape(-1, 1)
    x_pred = torch.from_numpy(x_pred_np).float()
    y_true = func(x_pred_np)
    y_pred = model(x_pred)
    x_np = x_pred.numpy().reshape(-1)
    y_pred_np = y_pred.detach().numpy().reshape(-1)
    y_true_np = y_true.reshape(-1)
    l2_diff = avg_l2_diff(y_pred_np, y_true_np).astype(float)
    l2_diff_tot_before.append(l2_diff)
    
    
#     rng = np.random.default_rng(seed)
#     res = np.empty([0])
#     w = np.power(y_pred_np - y_true_np, 2)
#     w = w / np.sum(w)
#     sz = w.shape[0]
#     freq = rng.multinomial(nsample, w)

#     for i in range(sz):
#         if freq[i]:
#             res = np.concatenate((res, [x_np[i]]), axis=0)
#     print(res)    
    
#     torch.manual_seed(seed)
#     random.seed(seed)
#     np.random.seed(seed)
#     torch.use_deterministic_algorithms(True)

#     model = FullModel()
#     x_train = x_train_orig
#     x_train = np.concatenate((x_train, res), axis=0)
#     print("After AL train size = ", x_train.shape[0])
#     y_train = func(x_train)
#     x_train_torch = torch.from_numpy(x_train).float()
#     y_train_torch = torch.from_numpy(y_train).float()
#     train_dataset = torch.utils.data.TensorDataset(x_train_torch, y_train_torch)
#     train_loader = torch.utils.data.DataLoader(
#         train_dataset, shuffle=True)
#     optimizer = torch.optim.SGD(model.parameters(), lr=0.02)
#     criterion = torch.nn.MSELoss()

#     for epoch in range(1, 2000):
#         train(model, optimizer, criterion, epoch)

#     x_pred_np = np.arange(-2.5, 2.5001, 0.0001).reshape(-1, 1)
#     x_pred = torch.from_numpy(x_pred_np).float() 
#     y_true = func(x_pred_np)
#     y_pred = model(x_pred)
#     x_np = x_pred.numpy().reshape(-1)
#     y_pred_np = y_pred.detach().numpy().reshape(-1)
#     y_true_np = y_true.reshape(-1)  
#     l2_diff = avg_l2_diff(y_pred_np, y_true_np).astype(float)
#     l2_diff_tot_after.append(l2_diff)

Orig train size =  193
Epoch: 40, Average loss:      0.00075523
Epoch: 80, Average loss:      0.00054278
Epoch: 120, Average loss:      0.00044488
Epoch: 160, Average loss:      0.00036991
Epoch: 200, Average loss:      0.00032052
Epoch: 240, Average loss:      0.00026815
Epoch: 280, Average loss:      0.00023101
Epoch: 320, Average loss:      0.00021171
Epoch: 360, Average loss:      0.00020731
Epoch: 400, Average loss:      0.00018216
Epoch: 440, Average loss:      0.00018216
Epoch: 480, Average loss:      0.00017778
Epoch: 520, Average loss:      0.00016917
Epoch: 560, Average loss:      0.00016065
Epoch: 600, Average loss:      0.00014675
Epoch: 640, Average loss:      0.00016595
Epoch: 680, Average loss:      0.00015850
Epoch: 720, Average loss:      0.00014632
Epoch: 760, Average loss:      0.00015157
Epoch: 800, Average loss:      0.00014502
Epoch: 840, Average loss:      0.00014595
Epoch: 880, Average loss:      0.00013983
Epoch: 920, Average loss:      0.00013640
Epoch: 960, A

In [7]:
print(l2_diff_tot_before)
print(l2_diff_tot_after)

[9.02670263093271e-05, 5.5010423303188614e-05, 0.0001624393785016679, 9.143436962650474e-05, 0.00024397209652918426, 0.00010891986862175248, 0.0002525390613661608, 0.0002326618576410996, 7.217395814373806e-05, 0.0001551015928568084, 0.00030466835706806247, 0.00017229983889778904, 0.0023069476985882447, 9.11318958189079e-05, 6.758135622609902e-05, 0.00025248165799039496, 8.727168496269771e-05, 0.0002712952309211347, 0.0011703754193346174, 9.060231588204366e-05]
[]
